# Transkriptupload 

Dies ist eine Programmierkonsole, die Ihre Transkripte automatisch auf Fehler prüft und in ein Format zur maschinellen Weiterverarbeitung umwandelt. Die folgenden Zellen können durch einen Klick auf das "Play"-Symbol aktiviert werden, das dort sichtbar wird, wo nun eine leere Klammer zu sehen ist, wenn Sie mit der Maus über die Zelle fahren oder diese anklicken. Bitte starten Sie mit der ersten Zelle und wählen Sie im Dialog in der Ausgabespalte, die sich unter der Zelle öffnet, die Datei aus, die Sie korrigieren möchten.

###Code ausführen (1): klicken Sie auf das "Play"-Symbol zwischen den eckigen Klammern

In [ ]:
from google.colab import files
upload = files.upload()

Saving ADG0642_02_02_tc_tab.ods to ADG0642_02_02_tc_tab.ods


# Transkriptkorrektur

Führen Sie nun die Transkriptkorrektur aus, indem Sie die nächste Zelle aktivieren. Nach einem erfolgreichem Durchlauf sollte sich ein Download-Dialog öffnen oder der Download des Zip-Ordners mit den Ergebnissen automatisch starten.

###Code ausführen (2): klicken Sie auf das "Play"-Symbol zwischen den eckigen Klammern

In [ ]:
###################################################
### Installing Dependencies and Clearing Output ###
###################################################

!pip install pyexcel_ods3
!pip install odfpy

try:
  !rm -rf 'output.zip'
except FileNotFoundError:
  print('Vorhandener Output-Ordner wurde entfernt oder existiert nicht.')

In [ ]:
################################################
### Defining Timecode Manipulation Functions ###
################################################


def timecode_median(tc1, tc2, framerate):

    difference = timecode_to_frames(tc2, framerate) - timecode_to_frames(tc1, framerate)
    if difference < 2:
        raise ValueError("Differenz zu gering, Bildung eines Medians nicht möglich: " + tc1 + ', ' + tc2)

    median = frames_to_timecode(timecode_to_frames(tc1, framerate) + (difference/2), framerate)

    return median


def timecode_median_multi(tc1, tc2, framerate, divisor, *startingpoint):
    difference = timecode_to_frames(tc2, framerate) - timecode_to_frames(tc1, framerate)
    if difference < divisor:
        raise ValueError("Differenz zu gering, Bildung eines Medians nicht möglich: " + tc1 + ', ' + tc2)

    median_multi = []

    if startingpoint:
        for i in range(startingpoint[0], divisor):
            tc_new = frames_to_timecode(timecode_to_frames(tc1, framerate) + (difference / divisor * i), framerate)
            median_multi.append(tc_new)
    else:
        for i in range(divisor):
            tc_new = frames_to_timecode(timecode_to_frames(tc1, framerate) + (difference/divisor * i), framerate)
            median_multi.append(tc_new)

    return median_multi


def timecode_to_frames(tc, framerate):
    minutes = int(tc[:2]) * 60
    seconds = (minutes + int(tc[3:5])) * 60
    frames = (seconds + int(tc[6:8])) * framerate + int(tc[9:])
    return frames

def frames_to_timecode(frames, framerate):
    tc_fr = int(frames % framerate)
    if tc_fr < 10:
        tc_fr = '0' + str(tc_fr)
    tc_s = int((frames / framerate) % 60)
    if tc_s < 10:
        tc_s = '0' + str(tc_s)
    tc_m = int(frames / framerate / 60 % 60)
    if tc_m < 10:
        tc_m = '0' + str(tc_m)
    tc_h = int(frames / framerate / 60 / 60)
    if tc_h < 10:
        tc_h = '0' + str(tc_h)

    return str(tc_h) + ':' + str(tc_m) + ':' + str(tc_s) + '.' + str(tc_fr)

In [ ]:
############################################
############################################
#### LUSIR TRANSCRIPT CORRECTION TOOL 3 ####
############################################
############################################




def lusir_transcript_correction_list_odfpy_3(source, file_raw):

    import re
    import io
    import os
    import pandas as pd
    from pandas import DataFrame
    from pyexcel_ods3 import save_data
    from collections import OrderedDict

    try:
      os.mkdir('OUTPUT')
    except FileExistsError:
      print('Ordner "OUTPUT" existiert bereits')
    logfile = ''
    auftragsbuch = ''
    file = pd.read_excel(file_raw, engine="odf")

############################################################
## Leere/falsche Zeilen und Spalten entfernen/korrigieren ##
############################################################

    file.reset_index(drop=True) # Index neu setzen

    try:
        file.pop('Übersetzung')
    except KeyError:
        print('Keine Spalte "Übersetzung"') # -> Logfile

    try:
        file.pop('Anmerkungen')
    except KeyError:
        print('Keine Spalte "Anmerkungen"') # -> Logfile

    for i in range(10):
        try:
            file.pop('Unnamed: '+ str(i))
        except KeyError:
            print('keine leere Spalte '+str(i))

##########################################################################
## Formatierung vereinheitlichen und Transkriptionszeichen herausparsen ##
##########################################################################
    try:
        file['Transkript'] = file['Transkript'].str.replace('{', '')
        file['Transkript'] = file['Transkript'].str.replace('}', '')
    except ValueError: # falls falsche Datentypen in den Zeilen versteckt sind, funktioniert die parallele Verarbeitung nicht
        file_list_parser = file.values.tolist()
        for line in file_list_parser:
            try:
                line[1] = line[1].replace('{', '').replace('}', '')
            except AttributeError: # falsche Datentypen überspringen, z.B. Zeitstempel, die keine Bedeutung haben
                continue
            except ValueError: # am Ende der gefüllten Spalten beenden (es folgen nan = floats)
                break
        file = DataFrame(file_list_parser, columns=['Timecode', 'Transkript'])
    except KeyError: # Spalte Transkript nicht benannt, Weiterverarbeitung nicht möglich
        auftragsbuch = auftragsbuch + 'Spalte Transkript nicht gefunden: ' + source + '\n'
        return auftragsbuch


    try:

        file['Timecode'] = file['Timecode'].str.replace(',', '.')
        file['Timecode'] = file['Timecode'].str.replace('[', '')
        file['Timecode'] = file['Timecode'].str.replace(']', '')

    except AttributeError: # in einigen dokumenten sind die Timecodes als Datetime-Objekt angegeben und müssen in einen String umgewandelt werden
        file_list = file.values.tolist()
        logfile = logfile + 'Timecode als Datetime in Excel\n'
        for id, line in enumerate(file_list):
            if type(line[0]) is float: # leere Zeilen (=nan =float) überspringen
                continue
            else:
                tc_new = [line[0].strftime('%H:%M:%S.%f')[:-4], line[1]]
                file_list.pop(id)
                file_list.insert(id, tc_new)
        file = DataFrame(file_list, columns=['Timecode', 'Transkript'])

    except KeyError: # Spalte Transkript nicht benannt, Weiterverarbeitung nicht möglich
        auftragsbuch = auftragsbuch + 'Spalte Timecode nicht gefunden: ' + source + '\n'
        return auftragsbuch

    file['Transkript'] = file['Transkript'].str.replace(':\*', '*') #Doppelpunkte vor Asterisk raus; Breakout vor Asterisk in Replace!!!
    file['Transkript'] = file['Transkript'].str.replace('\*:', '*') #Doppelpunkte nach Asterisk raus; Breakout vor Asterisk in Replace!!!


    if type(file.at[0, 'Timecode']) is float: # Wenn erster TC fehlt: 00:00:00.01 setzen
        file.at[0, 'Timecode'] = '00:00:00.01'
        logfile = logfile + 'Erster TC neu gesetzt\n'


    if type(file.at[0, 'Timecode']) is str and len(file.at[0, 'Timecode']) < 10: # Wenn erste Zeile ein Kolumnenindex ist: entfernen
        file.drop(0, inplace=True)
        logfile = logfile + 'Indexzeile gelöscht\n'

# -> DF neu indizieren?

##################################
## Dataframe in Liste umwandeln ##
##################################

    file_list = file.values.tolist()
    new_file_list = []

######################################
## Timecode-Chronologie korrigieren ##
######################################

    file_list_2 = []
    for line in file_list: # leere Zeilen entfernen
        if type(line[1]) is float:
            continue
        else:
            file_list_2.append(line)

    frames_set = 0
    for line in file_list_2: # Framerate bestimmen
        if type(line[0]) is str and not line[0].isspace():
            if line[0][0] == ' ':
                line[0] = line[0][1:]
            if int(line[0][9:12]) > frames_set:
                frames_set = int(line[0][9:12])
        else:
            continue

    if frames_set > 24:
        framerate = 30
    if frames_set == 24:
        framerate = 25
    if frames_set < 24:
        framerate = 24

    tc_frames_set = 0 # Timecodemarker setzen, um die Chronologie zu prüfen

    for id, line in enumerate(file_list_2): # über alle Zeilen iterieren, Timecode ist in erster (= nullter) Spalte
        tc = line[0]

        if id <= len(file_list_2) - 2: # der jeweils nächste Timecode zur Berechnung eines Medians, sollte einer fehlen
            next_tc = file_list_2[id+1][0]
        else:
            next_tc = next_tc

        if type(line[1]) is float or line[1].isspace(): # Zeilen ohne Transkription erkennen
            if type(line[0]) is str:
                auftragsbuch = auftragsbuch + 'Keine Transkription in Spalte: ' + source + str(line[0]) + '\n'
            continue

        if type(tc) is str: # wenn ein TC vorhanden, ist es ein String aus Zahlen
            frames = timecode_to_frames(tc, framerate) # scheinbar sind in manchen Dokumenten einstellige Timecodes ohne Null angegeben, das wirft error
            if frames <= tc_frames_set:     # außerdem: scheinbar Unicode Errors in manchen Doks in der TC-SPlate
                if timecode_to_frames(next_tc, framerate) > tc_frames_set:
                    try:
                        tc_new = timecode_median(frames_to_timecode(tc_frames_set, framerate), next_tc, framerate)
                        tc_frames_set = timecode_to_frames(tc_new, framerate)
                        new_file_list.append([tc_new, line[1]])
                        logfile = logfile + tc_new + " neu gesetzt (alt: " + frames_to_timecode(frames, framerate) + ')\n'
                    except ValueError:
                        auftragsbuch = auftragsbuch + "Differenz zu gering, Bildung eines Medians nicht möglich: " + source + ' ' + str(line[0]) + ', ' + file_list[id+1][0] + '\n'
                        continue
                elif timecode_to_frames(next_tc, framerate) <= tc_frames_set:
                    auftragsbuch = auftragsbuch + 'Mehrere falsche Timecodes in Folge: ' + source + ' ' + str(line[0]) + '\n'
                    return auftragsbuch
            if frames > tc_frames_set:
                tc_frames_set = frames
                new_file_list.append(line)

        if type(tc) is float: # wenn kein TC eingetragen, ist es ein NaN (dataframespezifisch) und das ist wiederum ein float (?!)
            if type(next_tc) is float and id != len(file_list_2):
                print('\n\nHIER:\n' + str(line) + '\n\n')
                auftragsbuch = auftragsbuch + 'Mehrere fehlende Timecodes in Folge: ' + source + ' ' + str(line[0]) + '\n'
                return auftragsbuch
            elif type(next_tc) is float and id == len(file_list_2):
                auftragsbuch = auftragsbuch + 'Letzter Timecode fehlt: ' + source + '\n'
                break
            else:
                tc_new = timecode_median(frames_to_timecode(tc_frames_set, framerate), next_tc, framerate)
                tc_frames_set = timecode_to_frames(tc_new, framerate)
                new_file_list.append([tc_new, line[1]])
                logfile = logfile + str(tc_new) + " neu gesetzt, vorher None\n"

############################################################################################################
## In Spalte Transkript: wenn Anzahl Sternchen eins: Fehlermeldung; wenn Anzahl Sternchen >2 und gerade:
## vor jedem ungeraden Sternchen cut und den folgenden Inhalt in eine neue Zeile einfügen,
############################################################################################################

###########################################################################
## Zunächst Sprecherliste für Dokument erstellen und ausgeben für Upload ##
###########################################################################

    file_list_final = []
    sprecher_doc = []
    for ID, line in enumerate(new_file_list):

        if len(re.findall(r"\*", line[1])) % 2 == 0: # wenn Anzahl Atserisks gerade
            sprecher_check = re.findall(r"\*([0-9A-Za-zäÄöÖüÜß. ]+?)\*", line[1]) # Mit Regex alle Sprecher in der Zeile finden
            for i in sprecher_check:
                if i not in sprecher_doc:
                    sprecher_doc.append(i) # Sprecherliste für Dokument aufbauen

        if len(re.findall(r"\*", line[1])) % 2 != 0: # Wenn anzahl Asterisks ungerade
            if not '<***>' in line[1]: # falls nicht Auszeichnung für Bandende (<***>) in Zeile
                for i in sprecher_doc:
                    if '*' + i + ' ' in line[1]:
                        line[1] = line[1].replace('*' + i, '*' + i + '*')  # schließendes Asterisk ergänzen
                        continue
                    if '*' + i + ':' in line[1]:
                        line[1] = line[1].replace('*' + i + ':', '*' + i + '*')  # schließendes Asterisk ergänzen, Doppelpunkt entfernen
                        continue
                    if i + '*' in line[1]:
                        line[1] = line[1].replace(i + '*', '*' + i + '*') # öffnendes Asterisk ergänzen
                        continue

    if len(sprecher_doc) < 2:
        print('Zu wenige Sprecher erkannt, bitte überprüfen!')
        auftragsbuch = auftragsbuch + 'Zu wenige Sprecher erkannt, bitte überprüfen: ' + source

    out = open('OUTPUT/' + source[:-4] + '_SPRECHER.txt', 'w', encoding='UTF-8')
    out.write(str(sprecher_doc))
    out.close()

    for ID, line in enumerate(new_file_list):
        if len(re.findall(r"\*", line[1])) % 2 == 0:  # wenn Anzahl Asterisks gerade

            sprecher = re.findall(r"\*([0-9A-Za-zäÄöÖüÜß. ]+?)\*", line[1])  # Mit Regex alle Sprecher in der Zeile finden
            sequenz = re.split(r"\*([0-9A-Za-zäÄöÖüÜß. ]+?)\*", line[1]) # Zeile nach den Sprechern aufsplitten und Ergebnis in Liste speichern

            if sequenz[0] == '':
                sequenz.pop(0)

            if len(sprecher) == 1 and sequenz[0] in sprecher: # Ein Sprecher, Sprecher steht am Anfang der Zeile
                file_list_final.append([ID, line[0], sequenz[0], sequenz[1]])
                continue

            elif len(sprecher) > 1 and sequenz[0] in sprecher: # Mehrere Sprecher, ein Sprecher steht am Anfang der Zeile
                timecodes = timecode_median_multi(line[0], new_file_list[ID + 1][0], framerate, len(sprecher))
                for i in range(len(sprecher)):
                    file_list_final.append([ID, timecodes[i], sequenz[0], sequenz[1]]) #<- Median: durch anzahl teilen und jew. 1/xtel mehr addieren?
                    sequenz.pop(0)
                    sequenz.pop(0)
                    logfile = logfile + 'Sprecherwechsel eingefügt: ' + timecodes[i] + '\n'
                continue

            elif len(sprecher) > 1 and sequenz[0] not in sprecher: # wenn mehrere Sprechwechsel, aber keiner am Anfang der Zeile
                file_list_final.append([ID, line[0], '§$%', sequenz[0]])
                sequenz.pop(0) # ersten Satz aus Sequenz entfernen
                timecodes = timecode_median_multi(line[0], new_file_list[ID + 1][0], framerate, len(sprecher)+1, 1)
                for i in range(len(sprecher)):  # <- funktioniert die for Schleife?
                    file_list_final.append([ID, timecodes[i], sequenz[0], sequenz[1]]) #<- Median: durch anzahl teilen und jew. 1/xtel mehr addieren?
                    sequenz.pop(0)
                    sequenz.pop(0)
                    logfile = logfile + 'Sprecherwechsel eingefügt: ' + timecodes[i] + '\n'
                continue

            elif len(sprecher) == 1 and sequenz[0] not in sprecher and sequenz[2] not in sprecher: # Ein Sprecher, Sprecher steht
                file_list_final.append([ID, line[0], '§$%', sequenz[0]])                            # mitten in der Zeile
                try:
                    file_list_final.append([ID, timecode_median(line[0], new_file_list[ID+1][0], framerate), sequenz[1], sequenz[2]]) # <- neuer Timecode: Median setzen!
                    logfile = logfile + 'Sprecherwechsel eingefügt: ' + timecode_median(line[0], new_file_list[ID+1][0], framerate) + '\n'
                except IndexError:
                    file_list_final.append([ID, '???', sequenz[1], sequenz[2]])
                    auftragsbuch = auftragsbuch + 'Letzten TC setzen: ' + source
                continue

            elif len(sprecher) == 0: # kein Sprecher in Zeile
                file_list_final.append([ID, line[0], '§$%', sequenz[0]]) # Problem könnte sein, dass auch leere Zeilen hier reinkommen
                continue

        if len(re.findall(r"\*", line[1])) % 2 != 0: # Wenn anzahl Asterisks immer noch ungerade

            if '<***>' in line[1] and len(re.findall(r"\*", line[1])) > 3:

                sprecher = re.findall(r"\*([0-9A-Za-zäÄöÖüÜß. ]+?)\*",
                                      line[1])  # Mit Regex alle Sprecher in der Zeile finden
                sequenz = re.split(r"\*([0-9A-Za-zäÄöÖüÜß. ]+?)\*",
                                   line[1])  # Zeile nach den Sprechern aufsplitten und Ergebnis in Liste speichern

                if sequenz[0] == '':
                    sequenz.pop(0)

                if len(sprecher) == 1 and sequenz[0] in sprecher:  # Ein Sprecher, Sprecher steht am Anfang der Zeile
                    file_list_final.append([ID, line[0], sequenz[0], sequenz[1]])
                    continue

                elif len(sprecher) > 1 and sequenz[
                    0] in sprecher:  # Mehrere Sprecher, ein Sprecher steht am Anfang der Zeile
                    timecodes = timecode_median_multi(line[0], new_file_list[ID + 1][0], framerate, len(sprecher))
                    for i in range(len(sprecher)):
                        file_list_final.append([ID, timecodes[i], sequenz[0], sequenz[
                            1]])  # <- Median: durch anzahl teilen und jew. 1/xtel mehr addieren?
                        sequenz.pop(0)
                        sequenz.pop(0)
                        logfile = logfile + 'Sprecherwechsel eingefügt: ' + timecodes[i] + '\n'
                    continue

                elif len(sprecher) > 1 and sequenz[
                    0] not in sprecher:  # wenn mehrere Sprechwechsel, aber keiner am Anfang der Zeile
                    file_list_final.append([ID, line[0], '§$%', sequenz[0]])
                    sequenz.pop(0)  # ersten Satz aus Sequenz entfernen
                    timecodes = timecode_median_multi(line[0], new_file_list[ID + 1][0], framerate, len(sprecher) + 1,
                                                      1)
                    for i in range(len(sprecher)):  # <- funktioniert die for Schleife?
                        file_list_final.append([ID, timecodes[i], sequenz[0], sequenz[
                            1]])  # <- Median: durch anzahl teilen und jew. 1/xtel mehr addieren?
                        sequenz.pop(0)
                        sequenz.pop(0)
                        logfile = logfile + 'Sprecherwechsel eingefügt: ' + timecodes[i] + '\n'
                    continue

                elif len(sprecher) == 1 and sequenz[0] not in sprecher and sequenz[
                    2] not in sprecher:  # Ein Sprecher, Sprecher steht
                    file_list_final.append([ID, line[0], '§$%', sequenz[0]])  # mitten in der Zeile
                    try:
                        file_list_final.append(
                            [ID, timecode_median(line[0], new_file_list[ID + 1][0], framerate), sequenz[1],
                             sequenz[2]])
                        logfile = logfile + 'Sprecherwechsel eingefügt: ' + timecode_median(line[0],
                                                                                            new_file_list[ID + 1][0],
                                                                                            framerate) + '\n'
                    except IndexError:
                        file_list_final.append([ID, '???', sequenz[1], sequenz[2]])
                        auftragsbuch = auftragsbuch + 'Letzten TC setzen: ' + source
                    continue

            if '<***>' in line[1] and len(re.findall(r"\*", line[1])) == 3:
                file_list_final.append([ID, line[0], '#', line[1]])
                continue
            else:
                file_list_final.append([ID, line[0], '', line[1]])
                auftragsbuch = auftragsbuch + 'ungerade Anzahl Asterisks, prüfen: ' + source + ' ' + str(line[0]) + '\n'
                continue

#########################################################
## Leerzeichen am Anfang der Transkriptzeile entfernen ##
## wenn am Anfang der Transkriptzeile eine Zahl steht, ##
## produziert das beim Schreiben des ODS einen Fehler, ##
## daher muss ein Zeichen davor gesetzt werden         ##
#########################################################

    for line in file_list_final:
        if len(line[3]) > 1:
            while line[3][0] == ' ':
                line[3] = line[3][1:]

######################################################################
## Chronologie anhand der Reihenfolge der ursprünglichen IDs prüfen ##
######################################################################

    limit = 0
    for count, line in enumerate(file_list_final):
        if limit < len(file_list_final)-1:
            if file_list_final[count+1][0] - line[0] > 1:
                auftragsbuch = auftragsbuch + 'Fehler in der Reihenfolge der IDs: ' + source + ', Zeile: ' + str(count + 2) + '\n\n' # -> Auftragsbuch
                limit = limit + 1
            else:
                limit = limit + 1
                continue

#########################
## Dataframe erstellen ##
#########################

    file_new = DataFrame(file_list_final, columns = ['Index', 'Timecode', 'Sprecher', 'Transkript'])

###################################################################################
## Leere Zeilen in Spalte Sprecher mit dem jeweils darüber liegenden Wert füllen ##
###################################################################################

    for id, line in enumerate(file_new['Sprecher']):
        if id > 0 and '§$%' in line:
            file_new.at[id, 'Sprecher'] = file_new.at[id-1, 'Sprecher']
        if id == 0 and '§$%' in line:
            auftragsbuch = auftragsbuch + 'Kein vorheriger Sprecher lokalisierbar, bitte 1. Sprecher einfügen: ' + source + '\n\n'

    file_new.pop('Index')


####################
## Datei ausgeben ##
####################

# korrigiertes Transkript

    odf_list = [file_new.columns.values.tolist()] + file_new.values.tolist()

    data = OrderedDict()  #
    data.update({"Sheet 1": odf_list})
    save_data('OUTPUT/' + source[:-4] + '_NEW.ods', data)

# logfile

    out = open('OUTPUT/' + source[:-4] + '_LOGFILE.txt', 'w', encoding='UTF-8')
    out.write(logfile)
    out.close()

    return auftragsbuch

In [ ]:
########################################################################################################################
########################################################################################################################
########################################################################################################################


import os


Auftragsbuch = '#############################################################################################################\n' \
               'Das ist eine Auflistung von Jobs, die händisch korrigiert werden müssen.\n\n' \
               'Timecodes: sollten mehrere falsche oder fehlende Timecodes aufeinander folgen, der letzte Timecode\n' \
               'fehlen oder ein neuer Timecode nicht errechnet werden, weil die Differenz zwischen dem vorherigen\n' \
               'und dem nächsten zu gering ist, kann keine automatische Timecodekorrektur angewandt werden.\n' \
               'In dem Fall muss das Ursprungsdokument, das angezeigt wird, zunächst händisch korrigiert und\n' \
               'dann erneut durch das Programm geschleift werden.\n\n' \
               'Sprecherwechsel: wenn kein Sprecher lokalisierbar war (markiert durch §$%),\n' \
               'bitte händisch in das neue Dokument eintragen. Wenn der Sprechwechsel aufgrund fehlender Sternchen nicht\n' \
               'zugeordnet werden konnte (markiert durch ###), bitte händisch im neuen Dokument nachtragen.\n\n' \
               'Fehler in der Reihenfolge der IDs: bitte neues mit altem Dokument abgleichen, ob Zeilen verschwunden sind\n' \
               '##############################################################################################################\n\n\n'


if len(upload) > 0:
  for key, item in upload.items():
    print(key)
    Auftragsbuch = Auftragsbuch + lusir_transcript_correction_list_odfpy_3(key, item)
    out_auftragsbuch = open('OUTPUT/Auftragsbuch.txt', 'w', encoding='UTF-8')
    out_auftragsbuch.write(Auftragsbuch)
    out_auftragsbuch.close()


  for file in os.listdir():
    print(file)
    if file.endswith('.ods'):
      os.remove(file)

  upload = {}
  !zip -r 'output.zip' 'OUTPUT'
  !rm -rf 'OUTPUT'
  files.download("output.zip")

else:
  print('Bitte laden Sie die zu bearbeitenden Transkripte in der ersten Zelle hoch!')

ADG0642_02_02_tc_tab.ods
keine leere Spalte 1
keine leere Spalte 2
keine leere Spalte 3
keine leere Spalte 4
keine leere Spalte 5
keine leere Spalte 6
keine leere Spalte 7
keine leere Spalte 8
keine leere Spalte 9
.config
ADG0642_02_02_tc_tab.ods
OUTPUT
sample_data
  adding: OUTPUT/ (stored 0%)
  adding: OUTPUT/ADG0642_02_02_tc_tab_NEW.ods (deflated 1%)
  adding: OUTPUT/ADG0642_02_02_tc_tab_SPRECHER.txt (stored 0%)
  adding: OUTPUT/ADG0642_02_02_tc_tab_LOGFILE.txt (stored 0%)
  adding: OUTPUT/Auftragsbuch.txt (deflated 55%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Kontrolle und Nachbesserungen

<b>SOLLTE KEIN DOWNLOAD STARTEN, MÜSSEN SIE VERMUTLICH DIE DATEIEN IN ZELLE 1 NEU HOCHLADEN!</b>

<i>Die Transkripte werden nach jedem Durchgang aus Datenschutzgründen und um Verwechslungen mit Duplikaten zu vermeiden gelöscht.</i><br><br>

Bitte entpacken Sie den Ordner und begutachten Sie die Ergebnisse.

Im Ordner finden Sie:

- das überarbeitete Transkript mit dem Zusatz "_NEW", wenn keine schwerwiegenden Fehler in Ihrer Ursprungsdatei enthalten waren.
- ein Auftragsbuch, in dem sie Aufgaben finden, die noch händisch nachgebessert werden müssen. 
  - Sollte die Datei "..._NEW" nicht enthalten sein, finden Sie im Auftragsbuch eine Erklärung (unter den häufigsten Ursachen sind beispielsweise, dass die Spalten in Ihrem Dokument nicht oder nicht korrekt benannt sind oder mehrere falsche Timecodes hintereinander gesetzt wurden). Wenn Sie diese schwerwiegenden Fehler in Ihrer Quelldatei behoben haben, beginnen Sie das Prozedere bitte von der ersten Zelle an von vorn.
- eine Tabelle mit allen gefundenen Sprechern. Die Tabelle gibt Hinweise darauf, ob es möglicherweise zu Flüchtigkeitsfehlern oder Inkonsistenzen in der Benennung der Sprecher gekommen ist und ob die Sprecherauszeichnung immer korrekt ausgeführt wurde.
- das Logfile, in dem Sie bei Interesse nachverfolgen können, welche Korrekturen das Tool automatisch durchgeführt hat.

Sobald alle händischen Korrekturen durchgeführt worden sind, senden Sie den vollständigen Ordner bitte an Herrn Keskin/laden Sie die Dateien bitte [hier] hoch. Bei Problemen mit dem Tool wenden Sie sich bitte an dennis.moebus@fernuni-hagen.de.